In [62]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

In [63]:
model_name = "roberta-large"

tokenizer = AutoTokenizer.from_pretrained(model_name, ignore_mismatched_sizes=True)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_c

In [64]:
lr = 2e-5
epochs =  6
batch_size = 5
max_seq_len = 75

test_frac = 0.1

In [65]:
import os

def set_seed(seed=106052):
    """Set seed for reproducibility.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed()

In [66]:
class CEFRDataset(Dataset):
    """Classification dataset, built on top of pytorch dataset object
    """
    
    def __init__(self, texts, labels):
        
        self.encoder = LabelEncoder()
        print(self.encoder.__dict__)
        self.texts = texts
        self.labels = self.encoder.fit_transform(labels)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        label = self.labels[index]
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=max_seq_len,
            truncation=True,
            return_tensors="pt",
        )
        encoded_text["input_ids"] = encoded_text["input_ids"].squeeze()
        encoded_text["attention_mask"] = encoded_text["attention_mask"].squeeze()
        label = torch.tensor(label)

        return {
            "input_ids": encoded_text["input_ids"],
            "attention_mask": encoded_text["attention_mask"],
            "labels": label,
        }

    def get_labels(self):
        return self.labels

In [67]:
def train(train_set, valid_set, epochs=10, warmup_size=0.1, lr=1e-3, batch_size=2):
    model = get_model(model_name)
    optim = AdamW(model.parameters(), lr=lr)
    scheduler = get_scheduler(
        optim, warmup_size, round(len(train_set) / batch_size * epochs)
    )
    training_args = get_training_args(epochs, batch_size)
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_set,
        eval_dataset=valid_set,
        optimizers=[optim, scheduler],
        compute_metrics=compute_accuracy
    )
    trainer.train()
    trainer.save_model()
    return trainer

In [68]:
def get_model(pretrained_checkpoint):
    model = AutoModelForSequenceClassification.from_pretrained(
        pretrained_checkpoint, num_labels=2, ignore_mismatched_sizes=True
    )
    return model.to(device)

In [69]:
os.environ["WANDB_DISABLED"] = "true"


def get_scheduler(optimizer, warmup_size, total_steps):
    scheduler = get_cosine_schedule_with_warmup(
        optimizer,
        num_warmup_steps=round(total_steps * warmup_size),
        num_training_steps=total_steps,
    )
    return scheduler


def get_training_args(epochs, batch_size):
    return TrainingArguments(
        output_dir="./b",
        num_train_epochs=epochs,
        per_device_train_batch_size=batch_size,
        logging_steps=50,
        fp16=False,
        evaluation_strategy="epoch",
        eval_accumulation_steps=1,
        report_to=None,
#         save_total_limit=1,
#         load_best_model_at_end=True,
        save_strategy = 'epoch'
    )


def compute_accuracy(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}



In [70]:
lr = 2e-5
epochs =  8
batch_size = 8
max_seq_len = 512

In [71]:
def split_valid(df, frac=0.1):
    
    val = pd.DataFrame()
    val["text"] = ""
    val["label"] = -1
    
    for i in df.label.unique():
        val = pd.concat([val, df[df.label == i].sample(frac=frac)])
        
    return df[~df.index.isin(val.index)].reset_index(drop=True) , val.reset_index(drop=True)

In [72]:
train_set_df = pd.read_csv("../input/covid19-tweet-classification-challenge-by-zindi/updated_train.csv")
train_set_df.drop("ID", axis=1, inplace=True)
train_set_df = train_set_df.reset_index(drop=True)
train_set_df.columns=["text","label"]
train_set_df = train_set_df[train_set_df.label != "-"]
train_set_df = train_set_df[["text", "label"]]

In [73]:
train_set_df.label.value_counts()

0    2746
1    2541
Name: label, dtype: int64

In [74]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

train_set_df.text = train_set_df.text.apply(lambda x: x.replace("\r", "").replace("\n", " "))

# extra_df = pd.read_csv("../input/frenchcefr/french_mike_june.csv")
# extra_df.columns = ["text", "label", "label_"]
# extra_df = extra_df[["text", "label"]]
# extra_df.text = extra_df.text.astype(str)
#train_set_df = pd.concat([train_set_df, extra_df]).reset_index(drop=True)

train_set_df, valid_set_df = split_valid(train_set_df)

In [75]:
train_set_df.label = le.fit_transform(train_set_df.label)
valid_set_df.label = le.transform(valid_set_df.label)

In [76]:
valid_set_df

,text,label
0,All empty during a show at the Max Stadium bef...,1
1,ki saru 41 donated 2cr to mumbai police in thi...,1
2,Soft water soft skin Looking for water softene...,1
3,Hospitals get paid more if patients listed as ...,1
4,Italian politician Vittorio Sgarbi reports in ...,1
...,...,...
524,Stephon Marbury says Larry Brown tried to kick...,0
525,He left it on the table Unfinished All the spo...,0
526,In these tough times the best way to grow is t...,0
527,poole Lib Dem Leader Fasts for Holy Ramadan in...,0


In [77]:
from tqdm import tqdm 

def predict(model, text):
    
    preds = []
    
    for i in tqdm(range(len(text))):
        tokenized = tokenizer(text[i:i+1], return_tensors="pt", truncation=True, max_length=512).to("cuda")
        pred = model(**tokenized)
        preds.append(pred.logits.argmax(-1).item())

    return preds

In [80]:
import gc
torch.cuda.empty_cache()
gc.collect()

7283

In [81]:
train_set = CEFRDataset(train_set_df["text"], train_set_df["label"])
valid_set = CEFRDataset(valid_set_df["text"], valid_set_df["label"])


trainer_second = train(train_set, valid_set, epochs=epochs, warmup_size=0.2, lr=lr, batch_size=batch_size)
model = trainer_second.model

{}
{}


loading configuration file https://huggingface.co/roberta-large/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/dea67b44b38d504f2523f3ddb6acb601b23d67bee52c942da336fa1283100990.94cae8b3a8dbab1d59b9d4827f7ce79e73124efa6bb970412cd503383a95f373
Model config RobertaConfig {
  "_name_or_path": "roberta-large",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.20.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-lar

Epoch,Training Loss,Validation Loss


RuntimeError: CUDA out of memory. Tried to allocate 128.00 MiB (GPU 0; 15.90 GiB total capacity; 14.86 GiB already allocated; 27.75 MiB free; 15.08 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
class CEFRDatasettest(Dataset):
    """Classification dataset, built on top of pytorch dataset object
    """
    
    def __init__(self, texts):
        
        self.encoder = LabelEncoder()
        print(self.encoder.__dict__)
        self.texts = texts

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, index):
        text = self.texts[index]
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=max_seq_len,
            truncation=True,
            return_tensors="pt",
        )
        encoded_text["input_ids"] = encoded_text["input_ids"].squeeze()
        encoded_text["attention_mask"] = encoded_text["attention_mask"].squeeze()

        return {
            "input_ids": encoded_text["input_ids"],
            "attention_mask": encoded_text["attention_mask"],
        }

    def get_labels(self):
        return self.labels

In [ ]:
test = pd.read_csv("../input/covid19-tweet-classification-challenge-by-zindi/updated_test.csv")

In [ ]:
preds=predict(model, test.text.to_list())

In [ ]:
sub=pd.read_csv("../input/covid19-tweet-classification-challenge-by-zindi/updated_ss.csv")
sub.target=preds
sub

In [ ]:
from IPython.display import FileLink
def create_submission(submission_file, submission_name):
    submission_file.to_csv(submission_name+".csv",index=False)
    return FileLink(submission_name+".csv")
create_submission(sub, "submission")

****